In [15]:
import torch
from torchvision import datasets,transforms as T,models
from torch.utils.data import DataLoader
import numpy as np 
from collections import OrderedDict
from torch import optim,nn
import matplotlib.pyplot as plt 
import torch.nn.functional as F 

In [16]:
train_dir = '/content/drive/My Drive/data/train'
valid_dir = '/content/drive/My Drive/data/valid'
test_dir = '/content/drive/My Drive/data/test'

In [17]:
train_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


valid_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transfrom = valid_transform

In [18]:
trainset = datasets.ImageFolder(train_dir,transform = train_transform)
validset = datasets.ImageFolder(valid_dir,transform = valid_transform)
testset = datasets.ImageFolder(test_dir,transform = test_transfrom)

In [19]:
trainloader = DataLoader(trainset, batch_size = 64,shuffle = True)
validloader = DataLoader(validset, batch_size = 64,shuffle = True)
testloader = DataLoader(testset, batch_size = 64,shuffle = True)

In [20]:
model = models.vgg19_bn(pretrained=True)

for param in model.parameters():
    param.requires_grad = False




classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 4096)),
                                         ('relu', nn.ReLU()),
                                         ('dropout',nn.Dropout(0.5)),
                                         ('fc2', nn.Linear(4096, 4096)),
                                         ('relu', nn.ReLU()),
                                         ('drop', nn.Dropout(0.5)),
                                         ('fc3', nn.Linear(4096, 3)), 
                                         ('output', nn.LogSoftmax(dim=1))]))

model.classifier = classifier

In [21]:
model.load_state_dict(torch.load('/content/drive/My Drive/SkinCancerModelVGG19_bn.pt'))
model.cuda()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [22]:
criterion = nn.NLLLoss()

In [23]:
model.eval()
train_acc = 0 
valid_acc = 0 
test_acc = 0 
test_loss = 0 
train_loss = 0
val_loss = 0

for images,labels in trainloader:
  images = images.cuda()
  labels = labels.cuda()
  logps = model(images)
  loss = criterion(logps,labels)
  train_loss += loss.item()*images.size(0)
  ps = torch.exp(logps)
  top_p, top_class = ps.topk(1, dim = 1)
  T_equals = top_class == labels.view(*top_class.shape)
  train_acc += torch.mean(T_equals.type(torch.FloatTensor))

for images,labels in validloader:
  images = images.cuda()
  labels = labels.cuda()
  logps = model(images)
  loss = criterion(logps,labels)
  val_loss += loss.item()*images.size(0)
  ps = torch.exp(logps)
  top_p,top_class = ps.topk(1, dim = 1)
  V_equals = top_class == labels.view(*top_class.shape)
  valid_acc += torch.mean(V_equals.type(torch.FloatTensor))

for images,labels in testloader:
  images = images.cuda()
  labels = labels.cuda()
  logps = model(images)
  loss = criterion(logps,labels)
  test_loss += loss.item()*images.size(0)
  ps = torch.exp(logps)
  top_p,top_class = ps.topk(1, dim = 1)
  t_equals = top_class == labels.view(*top_class.shape)
  test_acc += torch.mean(t_equals.type(torch.FloatTensor))

print("Overall Training Accuracy : {}\n".format(train_acc/len(trainloader)))
print("Overall Validation Accuracy : {}\n".format(valid_acc/len(validloader)))
print("Overall Test Accuracy : {}\n".format(test_acc/len(testloader)))
print("Overall Train Loss : {}\n".format(train_loss/len(trainloader.dataset)))
print("Overall Valid Loss : {}\n".format(val_loss/len(validloader.dataset)))
print("Overall Test Loss : {}\n".format(test_loss/len(testloader.dataset)))

Overall Training Accuracy : 0.9130859375

Overall Validation Accuracy : 0.7263257503509521

Overall Test Accuracy : 0.7072916626930237

Overall Train Loss : 0.2823573727607727

Overall Valid Loss : 0.6986135979493459

Overall Test Loss : 0.724037725130717

